### docker-compose installation

In [ ]:
$ curl -SL https://github.com/docker/compose/releases/download/v2.6.0/docker-compose-linux-x86_64 -o /usr/bin/docker-compose

In [ ]:
$ chmod +x /usr/bin/docker-compose

$ docker-compose version
Docker Compose version v2.6.0


$ file /usr/bin/docker-compose
/usr/bin/docker-compose: ELF 64-bit LSB executable, x86-64, version 1 (SYSV), statically linked, stripped

### docker-compose three tier geolocation deployment

In [ ]:
$ mkdir myapp

$ touch docker-compose.yml

In [ ]:
$ vim docker-compose.yml

services:
  
  redis:
    
    image: redis:latest
    container_name: redis
    restart: always
    networks:
      - geolocation-net


  backend:
    
    image: jinumona/ipgeolocation-api-redis:v2
    container_name: geolocation-api
    restart: always
    networks:
      - geolocation-net
    ports:
      - "81:8080"
    environment:
      - REDIS_PORT=6379
      - REDIS_HOST=redis
      - APP_PORT=8080
      - API_KEY=6a77a7aac63e4ed69a78f5d6c8cbe0f4

  frontend:
    
    image: jinumona/ipgeolocation-frontend:v2
    container_name: geolocation-frontend
    restart: always
    networks:
      - geolocation-net
    ports:
      - "80:8080"
    environment:
      - API_SERVER=geolocation-api
      - API_SERVER_PORT=8080
      - APP_PORT=8080

networks:
  geolocation-net:

In [ ]:
myapp]$ docker-compose config

name: myapp
services:
  backend:
    container_name: geolocation-api
    environment:
      API_KEY: 6a77a7aac63e4ed69a78f5d6c8cbe0f4
      APP_PORT: "8080"
      REDIS_HOST: redis
      REDIS_PORT: "6379"
    image: jinumona/ipgeolocation-api-redis:v2
    networks:
      geolocation-net: null
    ports:
    - mode: ingress
      target: 8080
      published: "81"
      protocol: tcp
    restart: always
  frontend:
    container_name: geolocation-frontend
    environment:
      API_SERVER: geolocation-api
      API_SERVER_PORT: "8080"
      APP_PORT: "8080"
    image: jinumona/ipgeolocation-frontend:v2
    networks:
      geolocation-net: null
    ports:
    - mode: ingress
      target: 8080
      published: "80"
      protocol: tcp
    restart: always
  redis:
    container_name: redis
    image: redis:latest
    networks:
      geolocation-net: null
    restart: always
networks:
  geolocation-net:
    name: myapp_geolocation-net


In [ ]:
$ docker-compose up -d

[+] Running 7/7
 ⠿ redis Pulled                                                                                                                                       5.8s
   ⠿ 42c077c10790 Already exists                                                                                                                      0.0s
   ⠿ a300d83d65f9 Pull complete                                                                                                                       1.0s
   ⠿ ebdc3afaab5c Pull complete                                                                                                                       1.1s
   ⠿ 6ce178c713e4 Pull complete                                                                                                                       1.7s
   ⠿ 949f9d8f429f Pull complete                                                                                                                       1.7s
   ⠿ 4076be5e5074 Pull complete                                                                                                                       1.8s
[+] Running 4/4
 ⠿ Network myapp_geolocation-net   Created                                                                                                            0.0s
 ⠿ Container geolocation-frontend  Started                                                                                                            2.1s
 ⠿ Container redis                 Started                                                                                                            1.6s
 ⠿ Container geolocation-api       Started                                                                                                            2.1s

In [ ]:
$ docker-compose down -v

[+] Running 4/4
 ⠿ Container geolocation-api       Removed                                                                                                            0.6s
 ⠿ Container geolocation-frontend  Removed                                                                                                            0.7s
 ⠿ Container redis                 Removed                                                                                                            0.5s
 ⠿ Network myapp_geolocation-net   Removed                                                                                                            0.0s

In [ ]:
$ docker-compose ps

NAME                   COMMAND                  SERVICE             STATUS              PORTS
geolocation-api        "python3 app.py"         backend             running             8080/tcp
geolocation-frontend   "python3 app.py"         frontend            running             0.0.0.0:80->8080/tcp, :::80->8080/tcp
redis                  "docker-entrypoint.s…"   redis               running             6379/tcp


In [ ]:
http://ec2-52-66-203-92.ap-south-1.compute.amazonaws.com/ip/8.8.8.8

In [ ]:
$ curl http://ec2-52-66-203-92.ap-south-1.compute.amazonaws.com:81/ip/8.8.8.8
        
{
  "apiServer": "98a226c589a6", 
  "cached": "True", 
  "calling_code": "+1", 
  "city": "Mountain View", 
  "connection_type": "", 
  "continent_code": "NA", 
  "continent_name": "North America", 
  "country_capital": "Washington, D.C.", 
  "country_code2": "US", 
  "country_code3": "USA", 
  "country_flag": "https://ipgeolocation.io/static/flags/us_64.png", 
  "country_name": "United States", 
  "country_tld": ".us", 
  "currency": {
    "code": "USD", 
    "name": "US Dollar", 
    "symbol": "$"
  }, 
  "district": "", 
  "geoname_id": "6301403", 
  "ip": "8.8.8.8", 
  "is_eu": false, 
  "isp": "Google LLC", 
  "languages": "en-US,es-US,haw,fr", 
  "latitude": "37.42240", 
  "longitude": "-122.08421", 
  "organization": "Google LLC", 
  "state_prov": "California", 
  "time_zone": {
    "current_time": "2022-06-21 06:46:20.762-0700", 
    "current_time_unix": 1655819180.762, 
    "dst_savings": 1, 
    "is_dst": true, 
    "name": "America/Los_Angeles", 
    "offset": -8
  }, 
  "zipcode": "94043-1351"
}

## Amazon ECR
Amazon Elastic Container Registry

In [ ]:
https://docs.aws.amazon.com/AmazonECR/latest/public/getting-started-cli.html

##### Creating container

In [ ]:
$ docker container  run -p 80:80 -d --name myapp httpd:alpine
Unable to find image 'httpd:alpine' locally
alpine: Pulling from library/httpd
2408cc74d12b: Pull complete 
473fdc809735: Pull complete 
25ecdd5e0572: Pull complete 
0438888bc4bf: Pull complete 
82f8d2440449: Pull complete 
a53853a13824: Pull complete 
Digest: sha256:f1225de544c16469ca7e5874ebcaa9aeb734db6e52facf11e9d74e039ffb679a
Status: Downloaded newer image for httpd:alpine
ddcd0388923743d224e143bbddc16ef6188d53394d48495e5cf90e15755cecab


$ docker container ls -a

CONTAINER ID   IMAGE          COMMAND              CREATED         STATUS         PORTS                               NAMES
ddcd03889237   httpd:alpine   "httpd-foreground"   8 seconds ago   Up 7 seconds   0.0.0.0:80->80/tcp, :::80->80/tcp   myapp

                    $ docker container exec -it myapp /bin/sh

/usr/local/apache2 # 
/usr/local/apache2/htdocs # cat index.html 
<center><h1>My ECR Site-Jinu</h1></center>

##### Making Image using commit

In [ ]:
$ docker commit myapp myapp:latest


    
$ docker image ls
REPOSITORY   TAG       IMAGE ID       CREATED         SIZE
myapp        latest    ffca9c33e480   5 seconds ago   54.9MB
httpd        alpine    148b7e25daf6   7 days ago      54.9MB


##### Create a public repository

In [ ]:
via front end

##### Authenticate to a public registry

In [ ]:

[ec2-user@ip-172-31-34-86 ~]$ aws ecr-public get-login-password --region us-east-1 | docker login --username AWS --password-stdin public.ecr.aws/o8l7k4k1
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded



In [ ]:
$ docker tag myapp:latest public.ecr.aws/o8l7k4k1/myapp:latest
        
$ docker image ls
REPOSITORY                      TAG       IMAGE ID       CREATED          SIZE
myapp                           latest    ffca9c33e480   38 minutes ago   54.9MB
public.ecr.aws/o8l7k4k1/myapp   latest    ffca9c33e480   38 minutes ago   54.9MB


In [ ]:
$ docker push public.ecr.aws/o8l7k4k1/myapp:latest
    
The push refers to repository [public.ecr.aws/o8l7k4k1/myapp]
9cdb10969c85: Pushed 
017c4576d3c0: Pushed 
975fcfa21dfa: Pushed 
6caeeb2cb1a8: Pushed 
ea96470c3118: Pushed 
6373f18a4f66: Pushed 
24302eb7d908: Pushed 
latest: digest: sha256:6a63e0f9c0667fe3f91802286b3c8abf3c9507c131e14208fc641a14b093b09c size: 1779

In [ ]:
$ docker container  run -p 80:80 -d --name myapp public.ecr.aws/o8l7k4k1/myapp:latest
        
Unable to find image 'public.ecr.aws/o8l7k4k1/myapp:latest' locally
latest: Pulling from o8l7k4k1/myapp
2408cc74d12b: Pull complete 
473fdc809735: Pull complete 
25ecdd5e0572: Pull complete 
0438888bc4bf: Pull complete 
82f8d2440449: Pull complete 
a53853a13824: Pull complete 
95137c8c0239: Pull complete 
Digest: sha256:6a63e0f9c0667fe3f91802286b3c8abf3c9507c131e14208fc641a14b093b09c
Status: Downloaded newer image for public.ecr.aws/o8l7k4k1/myapp:latest
74b3469bcf2a0221153abb4e76e702a24311bb8888fd00cf0b72225f6320f9df
       
$ docker container ls -a
CONTAINER ID   IMAGE                                  COMMAND              CREATED         STATUS         PORTS                               NAMES
74b3469bcf2a   public.ecr.aws/o8l7k4k1/myapp:latest   "httpd-foreground"   3 minutes ago   Up 3 minutes   0.0.0.0:80->80/tcp, :::80->80/tcp   myapp

## Docker swarm

In [ ]:
$ docker swarm init

Swarm initialized: current node (ljcwwjcjqtivduc2hppuaalkq) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join --token SWMTKN-1-5jdi0g76inm9591mtnolbk6ou090zyf0rneljh21wy39jzmkw6-2144o7v8stayjvhhqgh3d0f3s 172.31.34.86:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.

In [ ]:
$ docker swarm join-token worker

To add a worker to this swarm, run the following command:

    docker swarm join --token SWMTKN-1-5jdi0g76inm9591mtnolbk6ou090zyf0rneljh21wy39jzmkw6-2144o7v8stayjvhhqgh3d0f3s 172.31.34.86:2377

In [ ]:
$ docker swarm join-token manager

To add a manager to this swarm, run the following command:

    docker swarm join --token SWMTKN-1-5jdi0g76inm9591mtnolbk6ou090zyf0rneljh21wy39jzmkw6-9vzb60wjiy5ghd91j5xsyqfm4 172.31.34.86:2377


In [ ]:
$ docker node ls

ID                            HOSTNAME                                      STATUS    AVAILABILITY   MANAGER STATUS   ENGINE VERSION
ljcwwjcjqtivduc2hppuaalkq *   ip-172-31-34-86.ap-south-1.compute.internal   Ready     Active         Leader           20.10.13


### On worker1 and worker2

In [ ]:
$ docker swarm join --token SWMTKN-1-5jdi0g76inm9591mtnolbk6ou090zyf0rneljh21wy39jzmkw6-2144o7v8stayjvhhqgh3d0f3s 172.31.34.86:2377
    
This node joined a swarm as a worker.

##### On manager

In [ ]:
$ docker node ls

ID                            HOSTNAME                                       STATUS    AVAILABILITY   MANAGER STATUS   ENGINE VERSION
ljcwwjcjqtivduc2hppuaalkq *   ip-172-31-34-86.ap-south-1.compute.internal    Ready     Active         Leader           20.10.13
dupmpbug9qx4fgmzrdybbv5rd     ip-172-31-37-234.ap-south-1.compute.internal   Ready     Active                          20.10.13
we9w3fhr9djrt5p8p07hmyq5i     ip-172-31-47-139.ap-south-1.compute.internal   Ready     Active                          20.10.13


##### There is internal loadbalancing in swarm

### Service Creation

In [ ]:
$ docker service create \
--name web \
--replicas 2 \
-p 80:80 \
jinumona/rolling-hello:green

In [ ]:
$ docker container ls -a
CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS          PORTS     NAMES
030f4ef28c5a   jinumona/rolling-hello:green   "/docker-entrypoint.…"   24 seconds ago   Up 23 seconds             web.1.hjrggbbf1g8blaofhgpup4tjt

$ docker service ls
ID             NAME      MODE         REPLICAS   IMAGE                          PORTS
jqhbf9qq0o9n   web       replicated   2/2        jinumona/rolling-hello:green   *:80->80/tcp


##### Manager acting as worker

In [ ]:
manager
$ hostname
ip-172-31-34-86.ap-south-1.compute.internal

$ docker service ps web
ID             NAME      IMAGE                          NODE                                           DESIRED STATE   CURRENT STATE           ERROR     PORTS
hjrggbbf1g8b   web.1     jinumona/rolling-hello:green   ip-172-31-34-86.ap-south-1.compute.internal    Running         Running 2 minutes ago             
v9cstwya4nx8   web.2     jinumona/rolling-hello:green   ip-172-31-37-234.ap-south-1.compute.internal   Running         Running 2 minutes ago 

In [ ]:
#Eventhough worker2 has no container the site will be shown on worker2

at worker2
$ netstat -ntlp|grep 80
tcp6       0      0 :::80                   :::*                    LISTEN      3487/dockerd 

### service update for swarm

In [ ]:
$ docker service update --replicas 6 web

web
overall progress: 6 out of 6 tasks 
1/6: running   [==================================================>] 
2/6: running   [==================================================>] 
3/6: running   [==================================================>] 
4/6: running   [==================================================>] 
5/6: running   [==================================================>] 
6/6: running   [==================================================>] 
verify: Service converged 


In [ ]:
# at worker2

[root@ip-172-31-37-234 ec2-user]# docker container ls
CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS          PORTS     NAMES
dc1f785eb45a   jinumona/rolling-hello:green   "/docker-entrypoint.…"   2 minutes ago    Up 2 minutes              web.5.zahjr53fb95ih5g6hlor2qtww
ed3fe33ed29f   jinumona/rolling-hello:green   "/docker-entrypoint.…"   13 minutes ago   Up 13 minutes             web.2.v9cstwya4nx82aspt32hky7yw


In [ ]:
[root@ip-172-31-34-86 ec2-user]$ docker service ps web
ID             NAME      IMAGE                          NODE                                           DESIRED STATE   CURRENT STATE            ERROR     PORTS
hjrggbbf1g8b   web.1     jinumona/rolling-hello:green   ip-172-31-34-86.ap-south-1.compute.internal    Running         Running 14 minutes ago             
v9cstwya4nx8   web.2     jinumona/rolling-hello:green   ip-172-31-37-234.ap-south-1.compute.internal   Running         Running 14 minutes ago             
t7556kzm0odz   web.3     jinumona/rolling-hello:green   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running 2 minutes ago              
gv3hvtt5xg29   web.4     jinumona/rolling-hello:green   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running 2 minutes ago              
zahjr53fb95i   web.5     jinumona/rolling-hello:green   ip-172-31-37-234.ap-south-1.compute.internal   Running         Running 2 minutes ago              
zgq19l4d0jjn   web.6     jinumona/rolling-hello:green   ip-172-31-34-86.ap-south-1.compute.internal    Running         Running 2 minutes ago

#### self recreating containers

In [ ]:

$ docker container ls
CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS          PORTS     NAMES
dc1f785eb45a   jinumona/rolling-hello:green   "/docker-entrypoint.…"   2 minutes ago    Up 2 minutes              web.5.zahjr53fb95ih5g6hlor2qtww
ed3fe33ed29f   jinumona/rolling-hello:green   "/docker-entrypoint.…"   13 minutes ago   Up 13 minutes             web.2.v9cstwya4nx82aspt32hky7yw
[root@ip-172-31-37-234 ec2-user]# docker container ls -a
CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS          PORTS     NAMES
dc1f785eb45a   jinumona/rolling-hello:green   "/docker-entrypoint.…"   5 minutes ago    Up 5 minutes              web.5.zahjr53fb95ih5g6hlor2qtww
ed3fe33ed29f   jinumona/rolling-hello:green   "/docker-entrypoint.…"   17 minutes ago   Up 17 minutes             web.2.v9cstwya4nx82aspt32hky7yw
[root@ip-172-31-37-234 ec2-user]# docker stop dc1f785eb45a ed3fe33ed29f
dc1f785eb45a
ed3fe33ed29f
[root@ip-172-31-37-234 ec2-user]# docker rm  dc1f785eb45a ed3fe33ed29f
dc1f785eb45a
ed3fe33ed29f
[root@ip-172-31-37-234 ec2-user]# docker container ls -a
CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS         PORTS     NAMES
1ca2a66f9c2c   jinumona/rolling-hello:green   "/docker-entrypoint.…"   11 seconds ago   Up 5 seconds             web.2.kefh3quyvkyisdt5c99t1ky46
b21641cbb249   jinumona/rolling-hello:green   "/docker-entrypoint.…"   11 seconds ago   Up 5 seconds             web.5.vhqx6uw06x77lxwguirji16e

##### Updating image

In [ ]:
$ docker service update --image jinumona/rolling-hello:yellow web

In [ ]:
# New containers getting created

$ docker service update --image jinumona/rolling-hello:yellow web
    
web
overall progress: 6 out of 6 tasks 
1/6: running   [==================================================>] 
2/6: running   [==================================================>] 
3/6: running   [==================================================>] 
4/6: running   [==================================================>] 
5/6: running   [==================================================>] 
6/6: running   [==================================================>] 
verify: Service converged 
[root@ip-172-31-34-86 ec2-user]# docker service ps web
ID             NAME        IMAGE                           NODE                                           DESIRED STATE   CURRENT STATE             ERROR                       PORTS
ttpyrj5j7i8r   web.1       jinumona/rolling-hello:yellow   ip-172-31-34-86.ap-south-1.compute.internal    Running         Running 38 seconds ago                                
hjrggbbf1g8b    \_ web.1   jinumona/rolling-hello:green    ip-172-31-34-86.ap-south-1.compute.internal    Shutdown        Shutdown 39 seconds ago                               
h7d1h4556eb4   web.2       jinumona/rolling-hello:yellow   ip-172-31-37-234.ap-south-1.compute.internal   Running         Running 32 seconds ago                                
kefh3quyvkyi    \_ web.2   jinumona/rolling-hello:green    ip-172-31-37-234.ap-south-1.compute.internal   Shutdown        Shutdown 34 seconds ago                               
v9cstwya4nx8    \_ web.2   jinumona/rolling-hello:green    ip-172-31-37-234.ap-south-1.compute.internal   Shutdown        Failed 3 minutes ago      "task: non-zero exit (1)"   
qsrfu2fay1yi   web.3       jinumona/rolling-hello:yellow   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running 28 seconds ago                                
t7556kzm0odz    \_ web.3   jinumona/rolling-hello:green    ip-172-31-47-139.ap-south-1.compute.internal   Shutdown        Shutdown 29 seconds ago                               
q7za42m50ltb   web.4       jinumona/rolling-hello:yellow   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running 42 seconds ago                                
gv3hvtt5xg29    \_ web.4   jinumona/rolling-hello:green    ip-172-31-47-139.ap-south-1.compute.internal   Shutdown        Shutdown 44 seconds ago                               
zxn1ra9750cu   web.5       jinumona/rolling-hello:yellow   ip-172-31-37-234.ap-south-1.compute.internal   Running         Running 23 seconds ago                                
vhqx6uw06x77    \_ web.5   jinumona/rolling-hello:green    ip-172-31-37-234.ap-south-1.compute.internal   Shutdown        Shutdown 24 seconds ago                               
zahjr53fb95i    \_ web.5   jinumona/rolling-hello:green    ip-172-31-37-234.ap-south-1.compute.internal   Shutdown        Failed 3 minutes ago      "task: non-zero exit (1)"   
6d4pvqqnut34   web.6       jinumona/rolling-hello:yellow   ip-172-31-34-86.ap-south-1.compute.internal    Running         Running 48 seconds ago                                
zgq19l4d0jjn    \_ web.6   jinumona/rolling-hello:green    ip-172-31-34-86.ap-south-1.compute.internal    Shutdown        Shutdown 49 seconds ago

In [ ]:
$ docker service update --image jinumona/rolling-hello:blue web

In [ ]:
$ docker service rollback web

In [ ]:
$ docker service rm web